In [1]:
import networkx as nx
import matplotlib.pyplot as plt
import numpy as np
import json
import time
import datetime
import random
import bisect
import itertools

In [2]:
def read_json_file(filename):
    with open(filename) as f:
        js_graph = json.load(f) #, default={'sender': 'source'})
        _attrs = dict(source='sender', target='receiver', name='guid',
              key='guid', link='links')
    return nx.readwrite.node_link_graph(js_graph, directed=True, multigraph=False, attrs={'link': 'links', 'source': 'sender', 'target': 'receiver', 'key': 'guid', 'name': 'guid'} )

G = read_json_file("test_caveman_small.json")

In [3]:
print(nx.info(G))

Name: 
Type: DiGraph
Number of nodes: 30
Number of edges: 270
Average in degree:   9.0000
Average out degree:   9.0000


In [4]:
def selectRoot(G, weight_attr):
    cum_weights = [0]*G.number_of_nodes()
    nodes = [0]*G.number_of_nodes()
    for i,v in enumerate(G.nodes):  
        cum_weights[i] = (v, G.degree(v, weight=weight_attr))
        nodes[i] = v 
    
    items = sorted(cum_weights,reverse=True, key=lambda x: x[1])
    root = items[:1] 
    return root[0][0]

In [5]:
print(G.edges.data())

[(0, 1, {'weight': 1983}), (0, 2, {'weight': 3609}), (0, 3, {'weight': 8424}), (0, 4, {'weight': 7745}), (0, 5, {'weight': 6940}), (0, 6, {'weight': 4074}), (0, 7, {'weight': 7580}), (0, 8, {'weight': 1529}), (0, 9, {'weight': 2698}), (1, 0, {'weight': 1983}), (1, 2, {'weight': 118}), (1, 3, {'weight': 3900}), (1, 4, {'weight': 3982}), (1, 5, {'weight': 6736}), (1, 6, {'weight': 3140}), (1, 7, {'weight': 4093}), (1, 8, {'weight': 3873}), (1, 9, {'weight': 6210}), (1, 15, {'weight': 1201}), (2, 0, {'weight': 3609}), (2, 1, {'weight': 118}), (2, 3, {'weight': 4563}), (2, 4, {'weight': 7750}), (2, 5, {'weight': 7652}), (2, 6, {'weight': 3117}), (2, 7, {'weight': 1236}), (2, 8, {'weight': 2725}), (2, 9, {'weight': 36}), (3, 0, {'weight': 8424}), (3, 1, {'weight': 3900}), (3, 2, {'weight': 4563}), (3, 4, {'weight': 6670}), (3, 5, {'weight': 888}), (3, 6, {'weight': 8787}), (3, 7, {'weight': 7261}), (3, 8, {'weight': 6659}), (3, 14, {'weight': 4916}), (4, 0, {'weight': 7745}), (4, 1, {'weigh

In [10]:
def FBF(G, weight_attr, root, threshold): 
    tree = nx.DiGraph() 
    tree.add_node(root)  
    
    edge, neighbours, top = FBF_recursive(G, tree, weight_attr, None, root) 
    tree.add_edge(edge[0], edge[1], weight=edge[2])
    
    while tree.number_of_nodes() != G.number_of_nodes():
        edge, neighbours, top = FBF_recursive(G, tree, weight_attr, neighbours, top)
        tree.add_edge(edge[0], edge[1], weight=edge[2])
        
    print("before dense_component_extraction")
    print(nx.info(tree))
    tree = dense_component_extraction(tree, threshold, weight_attr)
    
    print("after dense_component_extraction")
    print(nx.info(tree))
    
    return tree

In [14]:
def FBF_recursive(G, tree, weight_attr, neighbours = None, lastAdded = None):  
    # pick a neightbour Vn+1 with a highest degree
    if neighbours == None:
        neighbours = []  
    neighbours.extend(G.degree(G.neighbors(lastAdded), weight=weight_attr)) 
         
    neighbours = [x for x in neighbours if x[0] not in tree.nodes] 
    neighbours = list(set(neighbours)) 
     
    top = sorted(neighbours,reverse=True, key=lambda x: x[1])[:1][0] 
    
    #print("top", sorted(neighbours,reverse=True, key=lambda x: x[1])[:1])
    #print("top[0]", top)
    #print(tree.nodes )
    #print("edges of top[0]", G.edges(top[0])) 
    edges = [e for e in G.edges(top[0], data='weight') if e[0] in tree.nodes or e[1] in tree.nodes]
    
    #print("edges ",edges)
      
    # pick a node to attach to, with highest degree
    vn_weight = []
    for e in edges:
        if e[0] != top:
            node = e[0]
        else:
            node = e[1] 
        vn_weight.extend([x for x in G.degree(G.neighbors(node), weight=weight_attr) if x[0] in tree.nodes]) 
     
    vn_weight = list(set(vn_weight))  
    other_end = sorted(vn_weight, reverse=True, key=lambda x: x[1])[:1][0]
    
    #print("vn_weight:", vn_weight) 
    #print("other_end:", other_end) 
    #print(top[0],"other_end:", other_end[0]) 
    
    edge = [e for e in edges if e[0] == top[0] and e[1] == other_end[0] or e[1] == top[0] and e[0] == other_end[0]][0]
    
    #print(edge)
    return edge, neighbours, top[0]


In [15]:
def dense_component_extraction(tree, threshold, weight_attr):
    # for each edge not in tree
    skipped_edges = set(G.edges.data(weight_attr, default=0)) - set(tree.edges.data(weight_attr, default=0))  
    
    # compute shortest path, keep adding the ones with the longest path
    items = []
    for edge in skipped_edges:
        length = nx.shortest_path_length(G, source=edge[0], target=edge[1], weight = weight_attr)
        items.append((edge[0], edge[1], edge[2], length))
    
    items = sorted(items,reverse=True, key=lambda x: x[3])
    #print("items", items)
    for item in items: 
        if tree.number_of_edges() >= threshold:
            continue
        tree.add_edge(item[0], item[1], weight=item[2])
        #print(item[0], item[1], item[2])
    
    return tree

In [16]:
weight = 'weight'
threshold = G.number_of_edges() / 2 
root = selectRoot(G, weight)
print("root: ", root)
 
current_time = time.time()
tree = FBF(G, weight, root, threshold)
print("FBF took", time.time()-current_time)


root:  10
(20, 10, 4292)
(29, 20, 9775)
(12, 10, 8321)
(19, 10, 4463)
(25, 29, 9933)
(14, 10, 8566)
(3, 14, 4916)
(4, 3, 6670)
(8, 4, 7957)
(26, 29, 8380)
(17, 12, 2585)
(0, 4, 7745)
(13, 10, 9334)
(16, 10, 8809)
(18, 10, 9087)
(7, 4, 9017)
(6, 4, 7351)
(5, 4, 1613)
(27, 29, 1895)
(9, 4, 6397)
(11, 10, 5139)
(24, 29, 7150)
(23, 10, 9174)
(21, 29, 7120)
(22, 29, 5664)
(28, 29, 6312)
(1, 4, 3982)
(2, 4, 7750)
(15, 10, 7492)
before dense_component_extraction
Name: 
Type: DiGraph
Number of nodes: 30
Number of edges: 29
Average in degree:   0.9667
Average out degree:   0.9667
after dense_component_extraction
Name: 
Type: DiGraph
Number of nodes: 30
Number of edges: 135
Average in degree:   4.5000
Average out degree:   4.5000
FBF took 0.04199504852294922
